## Interact with BiG-SLICE SQL Query result

In [ ]:
import sqlite3
import pandas as pd
from os import path
import json

## Download bigslice query from azure

In [ ]:
# load credentials
with open("../secret/blob-key.json") as f:
    az_account = json.load(f)
account_name = az_account["account_name"]
key = az_account["key"]

In [ ]:
# fetch from azure
container = "processed"
item_to_download = "bigslice/bigslice_query.zip"
target = "../data/bigslice_query.zip"
! az Storage blob download --account-key {key} --account-name {account_name} \
    --container-name {container} --file {target} --name {item_to_download} \
    > ../data/bigslice_query.log

In [ ]:
# extract data
! (cd ../data/ && unzip -f bigslice_query.zip)

## Interact with sql db via sqlite3

In [ ]:
# initiate connection
result_folder = "../data/full_run_result/"
conn = sqlite3.connect(path.join(result_folder, "reports/1/data.db"))

In [ ]:
# get all table name from sql
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
sql_table_names = [i[0] for i in cursor.fetchall()]
sql_table_names

In [ ]:
# convert table to pandas df
df_tables = []
for t in sql_table_names:
    df = pd.read_sql_query(f"select * from {t};", conn)
    df_tables.append(df)

### EDA

In [ ]:
# slice 1 example BGCs
df_input = df_tables[1]
df_input#[df_input.id == 1]

In [ ]:
df_input["genome_id"] = [path.basename(i) for i in df_input.loc[:, "orig_folder"]]

In [ ]:
df_input.loc[:, ["genome_id", "orig_filename"]].to_csv("../tables/bigscape_mapping.csv", index=False)

In [ ]:
# see 10 top hits to BGCs
df_gcf = df_tables[9]
df_gcf[df_gcf.bgc_id == 1]